In [36]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString, MultiLineString
import re
import geopandas as gpd
from shapely.geometry import Point
from shapely.strtree import STRtree
from collections import defaultdict
import geopandas as gpd
from shapely import shortest_line
# Load your lines


gas  = gpd.read_file(r"C:\dev\ews\lk\all\data\lk_gas.gpkg")

In [37]:
output = gpd.GeoDataFrame(columns=["GWR_EGID", "ARTZH", "Projektiert", "Adresse", "PLZ", "ORTSCHAFTSNAME", "ID", "GEBID", "eigentuemer", "status", "distance", "Koordinaten", "Lokationsart" ,"geometry"])

In [38]:
gas = gas.drop(columns=['fileid', 'obj_id','medium', 'datenherr','datenlieferant','letzte_aenderung','lagebestimmung','breite','objektart','letzte_lieferung','tid'])

In [39]:
gas_filtered = gas[~gas["eigentuemer"].isin(
    ["SBB Infrastruktur", "Migration ausstehend"]
)]

In [40]:
Adressen = gpd.read_file("C:\dev\ews\Adressen_GIS-ZH_-OGD/Adressen_GIS-ZH_-OGD.gpkg")

gebäude = gpd.read_file("C:\dev\ews\AV_MOpublic-_Bodenbedeckung_-OGD\AV_MOpublic-_Bodenbedeckung_-OGD.gpkg", layer="AVZH_BODENBEDECKUNG_F")

#Gas nur bestehende Gebäude
# proj_gebäude = gpd.read_file("C:\dev\ews\AV_MOpublic-_Bodenbedeckung_-OGD\AV_MOpublic-_Bodenbedeckung_-OGD.gpkg", layer="AVZH_BODENBEDECKUNG_PROJ_F")

liegeschaften = gpd.read_file("C:\dev\ews\AV_MOpublic-_Liegenschaften_-OGD\AV_MOpublic-_Liegenschaften_-OGD.gpkg", layer="AVZH_LIEGENSCHAFTEN_F")

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:8: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:8: SyntaxWarning: invalid escape sequence '\d'
C:\Users\rigli\AppData\Local\Temp\ipykernel_9608\1040749919.py:1: SyntaxWarning: invalid escape sequence '\d'
  Adressen = gpd.read_file("C:\dev\ews\Adressen_GIS-ZH_-OGD/Adressen_GIS-ZH_-OGD.gpkg")
C:\Users\rigli\AppData\Local\Temp\ipykernel_9608\1040749919.py:3: SyntaxWarning: invalid escape sequence '\d'
  gebäude = gpd.read_file("C:\dev\ews\AV_MOpublic-_Bodenbedeckung_-OGD\AV_MOpublic-_Bodenbedeckung_-OGD.gpkg", layer="AVZH_BODENBEDECKUNG_F")
C:\Users\rigli\AppData\Local\Temp\ipykernel_9608\1040749919.py:8: SyntaxWarning: invalid escape sequence '\d'
  liegeschaften = gpd.read_file("C:\dev\ews\AV_MOpublic-_Liegenschaften_-OGD\AV_MOpublic-_Liegenschaften_-OGD.gpkg", layer="AVZH_LIEGENSCHAF

In [41]:
#remove useless attributes
liegeschaften.drop(columns=['OBJID','NBIDENT','NUMMER','VOLLSTAENDIGKEIT','FLAECHENMASS','BFSNR','BEARBEITUNGSDATUM','SHAPE.AREA','SHAPE.LEN'], inplace=True)


#filter only gebäude, remove useless attributes
gebäude = gebäude[(gebäude['ART'] == 'Gebäude')]
# proj_gebäude = proj_gebäude[proj_gebäude['ARTZHID'].isin([0, 1, 2, 3, 4, 5, 6, 7])]

gebäude.drop(columns=['OBJID','QUALITAET','ART','BEARBEITUNGSDATUM','ARTCHID','ARTZHID','GVZNUMMER','SHAPE.AREA','SHAPE.LEN'], inplace=True)
# proj_gebäude.drop(columns=['OBJID','QUALITAET','ART','BEARBEITUNGSDATUM','ARTCHID','ARTZHID','GVZNUMMER','SHAPE.AREA','SHAPE.LEN','BEWILLIGUNGSSTATUS'], inplace=True)

In [42]:
gebäude['Projektiert'] = 'Nein'
# proj_gebäude['Projektiert'] = 'Ja'

In [43]:
all_gebäude = gebäude

# all_gebäude = pd.concat([gebäude, proj_gebäude])

# all_gebäude =gpd.GeoDataFrame(all_gebäude, geometry='geometry')


In [44]:
all_gebäude["Koordinaten"] = all_gebäude.geometry.apply(
    lambda geom: f"{round(geom.exterior.coords[0][0])} {round(geom.exterior.coords[0][1])}"
)

all_gebäude['GEBID'] = range(len(all_gebäude))


In [45]:

# Create GeoDataFrame for first vertices
first_vertices = gpd.GeoDataFrame(
    gas_filtered.drop(columns='geometry'),  # keep other columns if needed
    geometry=gas_filtered.geometry.apply(lambda line: Point(line.coords[0])),
    crs=gas_filtered.crs
)

# Create GeoDataFrame for last vertices
last_vertices = gpd.GeoDataFrame(
    gas_filtered.drop(columns='geometry'),  # keep other columns if needed
    geometry=gas_filtered.geometry.apply(lambda line: Point(line.coords[-1])),
    crs=gas_filtered.crs
)


# Combine into one GeoDataFrame if desired
all_vertices = gpd.GeoDataFrame(
    pd.concat([first_vertices, last_vertices], ignore_index=True),
    crs=gas_filtered.crs
)



In [46]:
# Drop duplicate geometries
unique_vertices = all_vertices.drop_duplicates(subset='geometry')

In [47]:
points = unique_vertices

# Buffer points by 5 cm (0.05 meters)
points['buffered'] = points.geometry.buffer(0.05)

# Create a GeoDataFrame from buffered geometries for spatial join
buffer_gdf = gpd.GeoDataFrame(points.drop(columns='geometry'), geometry=points['buffered'], crs=points.crs)

# Spatial join: find which buffered points intersect which lines
join = gpd.sjoin(buffer_gdf, gas, how='left', predicate='intersects')

# Count number of unique lines each buffered point intersects
counts = join.groupby(join.index).size()

# Filter points that intersect less than 2 lines (keep those)
keep_points = counts[counts < 2].index

# Keep original points corresponding to these indices
anschlüsse = points.loc[keep_points]

print(f"Original points count: {len(points)}")
print(f"Filtered points count (intersect < 2 lines): {len(anschlüsse)}")

c:\dev\GWR\.venv\Lib\site-packages\geopandas\geodataframe.py:1969: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Original points count: 242279
Filtered points count (intersect < 2 lines): 71040


In [48]:
# Drop the extra geometry column ('buffered') before saving
anschlüsse = anschlüsse.drop(columns='buffered', errors='ignore')
# Make sure the active geometry is 'geometry' column
anschlüsse.set_geometry('geometry', inplace=True)
anschlüsse['ID'] = range(len(anschlüsse))
anschlüsse.to_file("gas_output.gpkg", layer='GASANSCHLUSS_P')
anschlüsse

,eigentuemer,status,geometry,ID
12,unbekannt,in_Betrieb,POINT (2677209.962 1245997.197),0
13,unbekannt,in_Betrieb,POINT (2677229.811 1246051.841),1
14,unbekannt,in_Betrieb,POINT (2677262.797 1246074.426),2
36,Stadtwerk Winterthur,in_Betrieb,POINT (2698456.216 1261370.705),3
107,Stadtwerk Winterthur,in_Betrieb,POINT (2695430.179 1261275.128),4
...,...,...,...,...
398328,nicht-erfasst,in_Betrieb,POINT (2677593.837 1250506.294),71035
398331,nicht-erfasst,in_Betrieb,POINT (2677589.187 1250502.623),71036
398332,SWISSGAS,in_Betrieb,POINT (2672575.59 1247563.664),71037
398334,SWISSGAS,ausser_Betrieb,POINT (2674780.658 1246933.057),71038


In [49]:
Adressen['Adresse'] = Adressen['STRASSENNAME'] + ' ' + Adressen['HAUSNUMMER']
Adressen_clean = Adressen[Adressen['GWR_EGID'].notna()]
Adressen_subset = Adressen_clean[['Adresse','GWR_EGID','PLZ','ORTSCHAFTSNAME']]

In [50]:
gebäude_Adressen = all_gebäude.merge(Adressen_subset, how='left', on='GWR_EGID')

In [ ]:
filtered_gebäude = gebäude_Adressen[
    ~(
        (gebäude_Adressen["Adresse"].isna() &
         (gebäude_Adressen["ARTZH"] == "Nebengebäude")))
]


# filtered_gebäude = gebäude_Adressen[
#     ~(
#         (gebäude_Adressen["Adresse"].isna() &
#          (gebäude_Adressen["ARTZH"] == "Nebengebäude"))
#         |
#         (gebäude_Adressen["GWR_EGID"].isna())
#     )
# ]


In [ ]:
def filter_out_number_dot_number(address):
    # Ensure it's a string before applying regex
    address = str(address)
    return not re.search(r'(?:\d+)?\.\d', address)
    
    
valid_gebäude = filtered_gebäude[filtered_gebäude['Adresse'].apply(filter_out_number_dot_number)]



valid_gebäude.to_file("gas_output.gpkg", layer='ALLE_VALIDE_GEBÄUDE_F')
liegeschaften.to_file("gas_output.gpkg", layer='LIEGENSCHAFTEN_F')

In [52]:
#spatial join with gebäude
gebäude_anschluss = valid_gebäude.sjoin(anschlüsse, how='inner', predicate='intersects')

#define Lokationsart
gebäude_anschluss['Lokationsart'] = 'Anschluss in Gebäude'
gebäude_anschluss

,GWR_EGID,BFSNR,ARTZH,geometry,Projektiert,Koordinaten,GEBID,Adresse,PLZ,ORTSCHAFTSNAME,index_right,eigentuemer,status,ID,Lokationsart
340,117968.0,243,Gebäude Wohnen,"POLYGON ((2672144.705 1250363.746, 2672144.854...",Nein,2672145 1250364,337,Hätschenstrasse 21,8953.0,Dietikon,235971,unbekannt,in_Betrieb,18507,Anschluss in Gebäude
358,118164.0,243,Gebäude Wohnen,"POLYGON ((2672102.438 1251026.117, 2672100.613...",Nein,2672102 1251026,355,Steinmürlistrasse 25,8953.0,Dietikon,234273,unbekannt,in_Betrieb,18116,Anschluss in Gebäude
377,117701.0,243,Gebäude Wohnen,"POLYGON ((2672107.178 1251375.355, 2672116.057...",Nein,2672107 1251375,374,Bleicherstrasse 53,8953.0,Dietikon,234686,unbekannt,in_Betrieb,18203,Anschluss in Gebäude
472,116950.0,243,Gebäude Wohnen,"POLYGON ((2672464.804 1249814.634, 2672465.259...",Nein,2672465 1249815,468,Bremgartnerstrasse 118,8953.0,Dietikon,35994,unbekannt,tot,1561,Anschluss in Gebäude
492,116851.0,243,Gebäude Wohnen,"POLYGON ((2672515.048 1250277.6, 2672514.849 1...",Nein,2672515 1250278,488,Haldenstrasse 3,8953.0,Dietikon,233477,unbekannt,tot,17922,Anschluss in Gebäude
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350579,3018646.0,118,Gebäude Wohnen,"POLYGON ((2708399.135 1236001.137, 2708399.669...",Nein,2708399 1236001,338577,Kehrwiesenstrasse 4,8630.0,Rüti ZH,329144,GWR,in_Betrieb,51193,Anschluss in Gebäude
350582,46918.0,118,Gebäude Wohnen,"POLYGON ((2708482.471 1235998.821, 2708482.816...",Nein,2708482 1235999,338580,Fägswilerstrasse 40,8630.0,Rüti ZH,329654,GWR,ausser_Betrieb,51326,Anschluss in Gebäude
350586,46927.0,118,Gebäude Wohnen,"POLYGON ((2708526.06 1236046.738, 2708527.243 ...",Nein,2708526 1236047,338584,Fägswilerstrasse 51,8630.0,Rüti ZH,326996,GWR,in_Betrieb,50451,Anschluss in Gebäude
350588,46933.0,118,Gebäude Wohnen,"POLYGON ((2708560.057 1236041.444, 2708554.414...",Nein,2708560 1236041,338586,Fägswilerstrasse 55,8630.0,Rüti ZH,329460,GWR,in_Betrieb,51278,Anschluss in Gebäude


In [53]:

output = gpd.GeoDataFrame(pd.concat([output, gebäude_anschluss[["GWR_EGID", "ARTZH", "Projektiert", "Adresse", "PLZ", "ORTSCHAFTSNAME", "ID", "GEBID", "eigentuemer", "status", "Koordinaten", "Lokationsart" ,"geometry"]]], ignore_index=True))


C:\Users\rigli\AppData\Local\Temp\ipykernel_9608\3388644291.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output = gpd.GeoDataFrame(pd.concat([output, gebäude_anschluss[["GWR_EGID", "ARTZH", "Projektiert", "Adresse", "PLZ", "ORTSCHAFTSNAME", "ID", "GEBID", "eigentuemer", "status", "Koordinaten", "Lokationsart" ,"geometry"]]], ignore_index=True))


In [54]:
output

,GWR_EGID,ARTZH,Projektiert,Adresse,PLZ,ORTSCHAFTSNAME,ID,GEBID,eigentuemer,status,distance,Koordinaten,Lokationsart,geometry
0,117968.0,Gebäude Wohnen,Nein,Hätschenstrasse 21,8953.0,Dietikon,18507,337,unbekannt,in_Betrieb,NaN,2672145 1250364,Anschluss in Gebäude,"POLYGON ((2672144.705 1250363.746, 2672144.854..."
1,118164.0,Gebäude Wohnen,Nein,Steinmürlistrasse 25,8953.0,Dietikon,18116,355,unbekannt,in_Betrieb,NaN,2672102 1251026,Anschluss in Gebäude,"POLYGON ((2672102.438 1251026.117, 2672100.613..."
2,117701.0,Gebäude Wohnen,Nein,Bleicherstrasse 53,8953.0,Dietikon,18203,374,unbekannt,in_Betrieb,NaN,2672107 1251375,Anschluss in Gebäude,"POLYGON ((2672107.178 1251375.355, 2672116.057..."
3,116950.0,Gebäude Wohnen,Nein,Bremgartnerstrasse 118,8953.0,Dietikon,1561,468,unbekannt,tot,NaN,2672465 1249815,Anschluss in Gebäude,"POLYGON ((2672464.804 1249814.634, 2672465.259..."
4,116851.0,Gebäude Wohnen,Nein,Haldenstrasse 3,8953.0,Dietikon,17922,488,unbekannt,tot,NaN,2672515 1250278,Anschluss in Gebäude,"POLYGON ((2672515.048 1250277.6, 2672514.849 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37336,3018646.0,Gebäude Wohnen,Nein,Kehrwiesenstrasse 4,8630.0,Rüti ZH,51193,338577,GWR,in_Betrieb,NaN,2708399 1236001,Anschluss in Gebäude,"POLYGON ((2708399.135 1236001.137, 2708399.669..."
37337,46918.0,Gebäude Wohnen,Nein,Fägswilerstrasse 40,8630.0,Rüti ZH,51326,338580,GWR,ausser_Betrieb,NaN,2708482 1235999,Anschluss in Gebäude,"POLYGON ((2708482.471 1235998.821, 2708482.816..."
37338,46927.0,Gebäude Wohnen,Nein,Fägswilerstrasse 51,8630.0,Rüti ZH,50451,338584,GWR,in_Betrieb,NaN,2708526 1236047,Anschluss in Gebäude,"POLYGON ((2708526.06 1236046.738, 2708527.243 ..."
37339,46933.0,Gebäude Wohnen,Nein,Fägswilerstrasse 55,8630.0,Rüti ZH,51278,338586,GWR,in_Betrieb,NaN,2708560 1236041,Anschluss in Gebäude,"POLYGON ((2708560.057 1236041.444, 2708554.414..."


In [55]:
gebäude_anschluss.to_file("gas_output.gpkg", layer='ANSCHLUSS_IN_GEBÄUDE_F')

In [56]:
not_geb_joined = anschlüsse[~anschlüsse['ID'].isin(output['ID'])]
not_geb_joined_geb = valid_gebäude[~valid_gebäude['GEBID'].isin(output['GEBID'])]

In [57]:

# ─────────────────────────────────────────────────────────────
# Step 1: Spatial join to get parcel ID for points and buildings

anschluss_liegenschaft = gpd.sjoin(
    not_geb_joined, liegeschaften, how="left", predicate='within'
)

# Save original geometry and use centroids for spatial join
not_geb_joined_geb['orig_geom'] = not_geb_joined_geb.geometry
not_geb_joined_geb['geometry'] = not_geb_joined_geb['orig_geom'].centroid

gebäude_liegenschaft = gpd.sjoin(
    not_geb_joined_geb, liegeschaften, how="left", predicate='within'
)

# Restore original geometry
gebäude_liegenschaft['geometry'] = gebäude_liegenschaft['orig_geom']
gebäude_liegenschaft = gebäude_liegenschaft.drop(columns=['orig_geom'])



c:\dev\GWR\.venv\Lib\site-packages\geopandas\geodataframe.py:1969: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [58]:

# Suppose ansch_gdf is your anschluss_liegenschaft GeoDataFrame
# and buildings_gdf is gebäude_liegenschaft

# Optionally, filter only matching parcels first (to reduce candidates)
# For example, do a spatial join or filter buildings by parcel.

def find_best_building_with_line(point_geom, parcel_id, buildings_gdf):
    # Filter buildings in the same parcel (or pre-indexed)
    candidates = buildings_gdf[buildings_gdf['EGRIS_EGRID'] == parcel_id]
    if candidates.empty:
        return None, None  # no match

    best_dist = float('inf')
    best_building = None
    best_line = None

    for _, b in candidates.iterrows():
        b_geom = b.geometry
        # compute shortest line between point and building
        line = shortest_line(point_geom, b_geom)
        d = line.length
        if d < best_dist:
            best_dist = d
            best_line = line
            best_building = b

    return best_building, best_line

# Now apply in a loop (or vectorized) to your anschluss_gdf
results = []
for _, a in anschluss_liegenschaft.iterrows():
    p = a.geometry
    pid = a['EGRIS_EGRID']
    b_row, line = find_best_building_with_line(p, pid, gebäude_liegenschaft)

    # Skip if no building found
    if b_row is None:
        continue

    results.append({
    'ansch_id': a.name,
    'eigentuemer': a['eigentuemer'],
    'status': a['status'],
    'ID': a['ID'],
    'GWR_EGID': b_row['GWR_EGID'],
    'GEBID': b_row['GEBID'],
    'line': line,
    'distance': line.length if line is not None else None,
})

# Convert results to a GeoDataFrame if you like
nearest_building_line = gpd.GeoDataFrame(results, geometry='line', crs=gebäude_liegenschaft.crs)
nearest_building_line


,ansch_id,eigentuemer,status,ID,GWR_EGID,GEBID,line,distance
0,14,unbekannt,in_Betrieb,2,116177.0,259018,"LINESTRING (2677262.797 1246074.426, 2677262.7...",0.000145
1,36,Stadtwerk Winterthur,in_Betrieb,3,1162136.0,287594,"LINESTRING (2698456.216 1261370.705, 2698456.2...",0.000428
2,123,Stadtwerk Winterthur,in_Betrieb,5,1159811.0,156891,"LINESTRING (2696123.996 1262344.3, 2696123.989...",0.016133
3,128,Stadtwerk Winterthur,in_Betrieb,6,1160913.0,70010,"LINESTRING (2694134.23 1262360.032, 2694142.32...",8.319374
4,131,Stadtwerk Winterthur,in_Betrieb,7,1158971.0,284976,"LINESTRING (2696158.167 1263694.072, 2696158.1...",0.000589
...,...,...,...,...,...,...,...,...
20466,398325,SWISSGAS,in_Betrieb,71033,210211869.0,216688,"LINESTRING (2677593.176 1250504.412, 2677587.3...",36.246218
20467,398327,nicht-erfasst,in_Betrieb,71034,210211869.0,216688,"LINESTRING (2677595.697 1250505.661, 2677587.3...",35.521976
20468,398328,nicht-erfasst,in_Betrieb,71035,210211869.0,216688,"LINESTRING (2677593.837 1250506.294, 2677587.3...",34.508653
20469,398331,nicht-erfasst,in_Betrieb,71036,210211869.0,216688,"LINESTRING (2677589.187 1250502.623, 2677587.3...",37.606778


In [ ]:
nearest_building_line = (
    nearest_building_line
        .dropna(subset=['distance'])
        .loc[nearest_building_line['distance'] < 20]
)

nearest_building_line.to_file("gas_output.gpkg", layer='NÄCHSTES_GEBÄUDE_IN_PARZELLE_L')
nearest_building_line

,ansch_id,eigentuemer,status,ID,GWR_EGID,GEBID,line,distance
0,14,unbekannt,in_Betrieb,2,116177.0,259018,"LINESTRING (2677262.797 1246074.426, 2677262.7...",0.000145
1,36,Stadtwerk Winterthur,in_Betrieb,3,1162136.0,287594,"LINESTRING (2698456.216 1261370.705, 2698456.2...",0.000428
2,123,Stadtwerk Winterthur,in_Betrieb,5,1159811.0,156891,"LINESTRING (2696123.996 1262344.3, 2696123.989...",0.016133
3,128,Stadtwerk Winterthur,in_Betrieb,6,1160913.0,70010,"LINESTRING (2694134.23 1262360.032, 2694142.32...",8.319374
4,131,Stadtwerk Winterthur,in_Betrieb,7,1158971.0,284976,"LINESTRING (2696158.167 1263694.072, 2696158.1...",0.000589
...,...,...,...,...,...,...,...,...
20466,398325,SWISSGAS,in_Betrieb,71033,210211869.0,216688,"LINESTRING (2677593.176 1250504.412, 2677587.3...",36.246218
20467,398327,nicht-erfasst,in_Betrieb,71034,210211869.0,216688,"LINESTRING (2677595.697 1250505.661, 2677587.3...",35.521976
20468,398328,nicht-erfasst,in_Betrieb,71035,210211869.0,216688,"LINESTRING (2677593.837 1250506.294, 2677587.3...",34.508653
20469,398331,nicht-erfasst,in_Betrieb,71036,210211869.0,216688,"LINESTRING (2677589.187 1250502.623, 2677587.3...",37.606778


In [60]:

nearest_building_line_subset = nearest_building_line.drop(columns=['line','ansch_id'])

nearest_building = (
    gebäude_liegenschaft
    .merge(nearest_building_line_subset, on="GEBID")
    .rename(columns={"GWR_EGID_x": "GWR_EGID"})
    .drop(columns=["GWR_EGID_y"], errors="ignore")
)

nearest_building

,GWR_EGID,BFSNR,ARTZH,geometry,Projektiert,Koordinaten,GEBID,Adresse,PLZ,ORTSCHAFTSNAME,index_right,EGRIS_EGRID,eigentuemer,status,ID,distance
0,117940.0,243,Gebäude Wohnen,"POLYGON ((2672195.939 1250638.961, 2672196.493...",Nein,2672196 1250639,339,Steinmürlistrasse 62,8953.0,Dietikon,494,CH675577016191,unbekannt,tot,1128,0.000320
1,118406.0,243,Gebäude Wohnen,"POLYGON ((2671866.34 1251098.656, 2671861.965 ...",Nein,2671866 1251099,341,Rainackerstrasse 9,8953.0,Dietikon,49178,CH696177925582,unbekannt,in_Betrieb,1462,0.000100
2,117844.0,243,Gebäude Wohnen,"POLYGON ((2671941.537 1251066.104, 2671934.212...",Nein,2671942 1251066,345,Badstrasse 2,8953.0,Dietikon,350,CH607755926186,unbekannt,in_Betrieb,1453,0.000205
3,118170.0,243,Gebäude Wohnen,"POLYGON ((2671950.733 1250877.466, 2671945.967...",Nein,2671951 1250877,347,Schützenstrasse 34,8953.0,Dietikon,49184,CH759261775515,unbekannt,tot,1647,6.984831
4,118170.0,243,Gebäude Wohnen,"POLYGON ((2671950.733 1250877.466, 2671945.967...",Nein,2671951 1250877,347,Schützenstrasse 34,8953.0,Dietikon,49184,CH759261775515,unbekannt,tot,18530,0.000164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21854,46912.0,118,Gebäude Wohnen,"POLYGON ((2708371.481 1235931.382, 2708373.722...",Nein,2708371 1235931,338576,Fägswilerstrasse 27,8630.0,Rüti ZH,341614,CH989259067710,GWR,ausser_Betrieb,51323,2.066497
21855,46912.0,118,Gebäude Wohnen,"POLYGON ((2708371.481 1235931.382, 2708373.722...",Nein,2708371 1235931,338576,Fägswilerstrasse 27,8630.0,Rüti ZH,341614,CH989259067710,GWR,in_Betrieb,51324,0.000113
21856,210255185.0,118,Nebengebäude,"POLYGON ((2708644.518 1236011.516, 2708638.928...",Nein,2708645 1236012,338581,Goldbachstrasse 5b,8630.0,Rüti ZH,145192,CH459359770660,GWR,ausser_Betrieb,50338,15.398566
21857,210255185.0,118,Nebengebäude,"POLYGON ((2708644.518 1236011.516, 2708638.928...",Nein,2708645 1236012,338581,Goldbachstrasse 5b,8630.0,Rüti ZH,145192,CH459359770660,GWR,in_Betrieb,50432,15.523564


In [61]:

nearest_building['Lokationsart'] = 'Nächstes Gebäude in Parzelle'

nearest_building.to_file("gas_output.gpkg", layer='NÄCHSTES_GEBÄUDE_IN_PARZELLE_F')

output = gpd.GeoDataFrame(pd.concat([output, nearest_building[["GWR_EGID", "ARTZH", "Projektiert", "Adresse", "PLZ", "ORTSCHAFTSNAME", "ID", "GEBID", "eigentuemer", "status", "Koordinaten", "distance", "Lokationsart" ,"geometry"]]], ignore_index=True))


C:\Users\rigli\AppData\Local\Temp\ipykernel_9608\1992964496.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output = gpd.GeoDataFrame(pd.concat([output, nearest_building[["GWR_EGID", "ARTZH", "Projektiert", "Adresse", "PLZ", "ORTSCHAFTSNAME", "ID", "GEBID", "eigentuemer", "status", "Koordinaten", "distance", "Lokationsart" ,"geometry"]]], ignore_index=True))


In [62]:
not_parz_joined = anschlüsse[~anschlüsse['ID'].isin(output['ID'])]
not_parz_joined_geb = valid_gebäude[~valid_gebäude['GEBID'].isin(output['GEBID'])]

In [63]:
# all_building_in_liegenschaft = anschluss_liegenschaft.merge(gebäude_liegenschaft, on='EGRIS_EGRID')

In [64]:
# all_building_in_liegenschaft = all_building_in_liegenschaft.drop(columns=['geometry_x', 'index_right_x', 'index_right_y', 'EGRIS_EGRID'])

In [65]:
# # Set the geometry column
# all_building_in_liegenschaft = all_building_in_liegenschaft.set_geometry('geometry_y')

# # Rename it to 'geometry' if it's not already called that
# all_building_in_liegenschaft = all_building_in_liegenschaft.rename_geometry('geometry')

In [66]:
# all_building_in_liegenschaft.to_file("gas_output.gpkg",layer="ALLE_GEBÄUDE_IN_LIEGENSCHAFT_F")

# all_building_in_liegenschaft['Lokationsart'] = 'Alle Gebäude in der Liegenschaft'

# output = gpd.GeoDataFrame(pd.concat([output, all_building_in_liegenschaft[["GWR_EGID", "ARTZH", "Projektiert", "Adresse", "PLZ", "ORTSCHAFTSNAME", "tid", "eigentuemer", "Lokationsart", "geometry","ID"]]], ignore_index=True))

In [67]:

building_sindex = not_parz_joined_geb.sindex

def find_best_building_with_line(point_geom, buildings_gdf, sindex, search_radius=100):
    """
    Finds the closest building to a point using the shortest_line.
    Uses spatial index for fast lookup.
    
    Returns:
        - best matching building row (GeoSeries)
        - shortest line (LineString)
    """
    # Bounding box around the point
    bbox = point_geom.buffer(search_radius).bounds
    candidate_idx = list(sindex.intersection(bbox))

    if not candidate_idx:
        return None, None

    candidates = buildings_gdf.iloc[candidate_idx]

    best_dist = float('inf')
    best_building = None
    best_line = None

    for _, b in candidates.iterrows():
        b_geom = b.geometry
        line = shortest_line(point_geom, b_geom)
        d = line.length

        if d < best_dist:
            best_dist = d
            best_line = line
            best_building = b

    return best_building, best_line

# ------------------------------
# Apply to all points in anschluss_liegenschaft
# ------------------------------

results = []

for _, a in not_parz_joined.iterrows():
    p = a.geometry
    b_row, line = find_best_building_with_line(
        point_geom=p,
        buildings_gdf=not_parz_joined_geb,
        sindex=building_sindex,
        search_radius=100  # adjust as needed
    )

    # Skip if no building found
    if b_row is None:
        continue

    results.append({
    'ansch_id': a.name,
    'eigentuemer': a['eigentuemer'],
    'status': a['status'],
    'ID': a['ID'],
    'GWR_EGID': b_row['GWR_EGID'],
    'GEBID': b_row['GEBID'],
    'line': line,
    'distance': line.length if line is not None else None,
    })

# ------------------------------
# Convert to GeoDataFrame
# ------------------------------

nearest_building_line_all = gpd.GeoDataFrame(results, geometry='line', crs=gebäude_liegenschaft.crs)

In [ ]:
nearest_building_line_all = nearest_building_line_all[
    nearest_building_line_all['distance'].notna() &
    (nearest_building_line_all['distance'] < 10)
]

nearest_building_line_all.to_file("gas_output.gpkg", layer='NÄCHSTES_GEBÄUDE_L')


nearest_building_line_all_subset = nearest_building_line_all.drop(columns=['line','ansch_id'])

# nearest_building_all = valid_gebäude.merge(nearest_building_line_all_subset, on='GEBID')

nearest_building_all = (
    valid_gebäude
    .merge(nearest_building_line_all_subset, on="GEBID")
    .rename(columns={"GWR_EGID_x": "GWR_EGID"})
    .drop(columns=["GWR_EGID_y"], errors="ignore")
)

nearest_building_all['Lokationsart'] = 'Nächstes Gebäude in angrenzender Parzelle'

output = gpd.GeoDataFrame(pd.concat([output, nearest_building_all[["GWR_EGID", "ARTZH", "Projektiert", "Adresse", "PLZ", "ORTSCHAFTSNAME", "ID", "GEBID", "eigentuemer", "status", "Koordinaten", "distance", "Lokationsart" ,"geometry"]]], ignore_index=True))


nearest_building_all.to_file("gas_output.gpkg", layer='NÄCHSTES_GEBÄUDE_F')

In [ ]:
import datetime

today = datetime.date.today()


# output['GWR_EGID'] = output['GWR_EGID'].fillna(0).astype(int)
output = output.dropna(subset=['GWR_EGID'])
output['GWR_EGID'] = output['GWR_EGID'].astype(int)


output = output.rename(columns={"GWR_EGID": "EGID", "ORTSCHAFTSNAME": "Ort", "eigentuemer": "Netzbetreiber", "Koordinaten": "x/y-Koordinate", "status": "Leitungssatus", "distance": "Gebäudedistanz"})

output = output.drop("geometry", axis=1)

output.to_csv(f"gas_output_{today}.csv")

In [70]:
output

,EGID,ARTZH,Projektiert,Adresse,PLZ,ORTSCHAFTSNAME,ID,GEBID,Netzbetreiber,Leitungssatus,Gebäudedistanz,x/y-Koordinate,Lokationsart
0,117968,Gebäude Wohnen,Nein,Hätschenstrasse 21,8953.0,Dietikon,18507,337,unbekannt,in_Betrieb,NaN,2672145 1250364,Anschluss in Gebäude
1,118164,Gebäude Wohnen,Nein,Steinmürlistrasse 25,8953.0,Dietikon,18116,355,unbekannt,in_Betrieb,NaN,2672102 1251026,Anschluss in Gebäude
2,117701,Gebäude Wohnen,Nein,Bleicherstrasse 53,8953.0,Dietikon,18203,374,unbekannt,in_Betrieb,NaN,2672107 1251375,Anschluss in Gebäude
3,116950,Gebäude Wohnen,Nein,Bremgartnerstrasse 118,8953.0,Dietikon,1561,468,unbekannt,tot,NaN,2672465 1249815,Anschluss in Gebäude
4,116851,Gebäude Wohnen,Nein,Haldenstrasse 3,8953.0,Dietikon,17922,488,unbekannt,tot,NaN,2672515 1250278,Anschluss in Gebäude
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74777,46873,Gebäude Wohnen,Nein,Bachtelstrasse 22a,8630.0,Rüti ZH,5682,338406,GWR,ausser_Betrieb,23.585313,2708076 1235977,Nächstes Gebäude in angrezender Parzelle
74778,210254666,Nebengebäude,Nein,Dachseggstrasse 3a,8630.0,Rüti ZH,50490,338564,GWR,in_Betrieb,7.251718,2708424 1234600,Nächstes Gebäude in angrezender Parzelle
74779,210254643,Nebengebäude,Nein,Fägswilerstrasse 60b,8630.0,Rüti ZH,50286,338587,GWR,in_Betrieb,12.283964,2708592 1236125,Nächstes Gebäude in angrezender Parzelle
74780,210255224,Nebengebäude,Nein,Goldbachstrasse 26a,8630.0,Rüti ZH,51202,338714,GWR,ausser_Betrieb,26.379181,2708882 1235971,Nächstes Gebäude in angrezender Parzelle
